In [1]:
from typing import *
import os, sys
import time

from tqdm import tqdm
import numpy as np
import open3d as o3d
import open3d.visualization as vis

def load_gzip(path):
    import gzip, pickle
    with gzip.open(path, 'rb') as f:
        return pickle.load(f)
    
def load_json(path):
    import json
    with open(path, "r") as f:
        return json.load(f)
    
def extract_trianglemesh(volume, 
                         engine: str, 
                         surface_weight_thr: Optional[float] = None,
                         file_name=None):
    if engine == 'legacy':
        mesh = volume.extract_triangle_mesh()
        mesh.compute_vertex_normals()
        mesh.compute_triangle_normals()
        if file_name is not None:
            o3d.io.write_triangle_mesh(file_name, mesh)

    elif engine == 'tensor':
        mesh = volume.extract_triangle_mesh(
            weight_threshold=surface_weight_thr)
        mesh = mesh.to_legacy()

        if file_name is not None:
            o3d.io.write_triangle_mesh(file_name, mesh)

    return mesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
save_dir = 'dat/recorded/d415/'
captures = load_gzip(os.path.join(save_dir, 'captures.gzip'))
calibrations = load_json(os.path.join(save_dir, 'calibrations.json'))
calibrations = {k:np.array(v) for k,v in calibrations.items()}

In [3]:
# device = 'CUDA:0'
# voxel_size = 0.005
# block_resolution = 16
# block_count = 40000
# depth_scale = 1000.0
# depth_min = 0.1
# depth_max = 3.0
# odometry_distance_thr = 0.07
# trunc_voxel_multiplier = 8.0
# engine = 'tensor'
# surface_weight_thr = 3.0

device = 'CUDA:0'
voxel_size = 0.003
block_resolution = 16
block_count = 10000
depth_scale = 1000.0
depth_min = 0.1
depth_max = 1.0
odometry_distance_thr = 0.07
trunc_voxel_multiplier = 4.0
engine = 'tensor'
surface_weight_thr = 3.0




In [4]:
if isinstance(device, str):
    device = o3d.core.Device(device)

intrinsic = o3d.core.Tensor(calibrations['camera_matrix'])
T_frame_to_model = o3d.core.Tensor(np.identity(4))
model = o3d.t.pipelines.slam.Model(
    voxel_size, 
    block_resolution,
    block_count, 
    transformation=T_frame_to_model,
    device=device
)

depth_ref = o3d.t.geometry.Image(captures[0][1])
input_frame = o3d.t.pipelines.slam.Frame(depth_ref.rows, depth_ref.columns,
                                            intrinsic, device)
raycast_frame = o3d.t.pipelines.slam.Frame(depth_ref.rows,
                                            depth_ref.columns, intrinsic,
                                            device)

In [5]:
poses = []

for i, capture in tqdm(enumerate(captures)):
    color, depth = o3d.t.geometry.Image(capture[0]).to(device), o3d.t.geometry.Image(capture[1]).to(device)

    input_frame.set_data_from_image('depth', depth)
    input_frame.set_data_from_image('color', color)

    if i > 0:
        result = model.track_frame_to_model(input_frame, raycast_frame,
                                            depth_scale,
                                            depth_max,
                                            odometry_distance_thr)
        T_frame_to_model = T_frame_to_model @ result.transformation

    poses.append(T_frame_to_model.cpu().numpy())
    model.update_frame_pose(i, T_frame_to_model)
    model.integrate(input_frame, depth_scale, depth_max,
                    trunc_voxel_multiplier)
    model.synthesize_model_frame(raycast_frame, depth_scale,
                                    depth_min, depth_max,
                                    trunc_voxel_multiplier, False)

13it [00:00, 127.89it/s]

[Open3D WARNING] Property not found for color, return an empty tensor!


293it [00:01, 224.58it/s]


In [6]:
volume, poses = model.voxel_grid, poses

In [7]:
mesh = extract_trianglemesh(volume, engine=engine, surface_weight_thr=surface_weight_thr)

In [8]:
o3d.visualization.draw_geometries([mesh])